# A Metric for Situational Defensive Linemen Performance

#### Utilizing Long-Short-Term Memory Networks to Track Player Contribution to the Likelihood of Play Disruption


"*When you hang with a bunch of 300-pound linemen, you tend to find the places that are the greasiest and serve the most food." - Tom Brady*



# Introduction

When you visualize exemplary and breathtaking athleticism, our 300-pound friends on the defensive line are not typically the first to come to mind—but maybe they should be. While NFL quarterbacks are commonly praised for the amazing last-second touchdowns that frequent RedZone highlight reels, they complete a more impressive feat at the beginning of every play: taking the snap in front of the opposing defensive linemen who have no other goal than to hit, block, and all around obliterate the offense. If you're not convinced, the Buffalo Bills hit on Mike White during week 14 of the 2022-2023 season might change your mind.


<div>
<center><img src="https://imgur.com/YDyemNf.gif" width="500"/><center>
</div>

The importance of the defensive linemen to the game of football is severely understated, but some are certainly more influential on play trajectory than others. How, then, can we appropriately assess skill on the defensive line? **This paper introduces the Instantaneous Disruption Probability Increase (IDPI) metric, which measures differences in performance of pass rushers compared to other lineman in similar situations. A linemen’s ability to improve his team's probability of play disruption is used as the standard for performance.** Unlike metrics reliant on the number of individual disuptions a pass rusher makes, which tend to favor the defensive linemen with the most highlight hits, the IDPI metric identifies those who increase their entire team’s chances of pass disruption.

<div>
<center><img src="https://imgur.com/ZbSgcY4.jpeg" width="700"/><center>
</div>

# Methods

Many current assessments of defensive linemen rely on the simple number of individual disruptions (i.e., sack, hurry, etc.) that a lineman records. While this is simple to compute, it doesn’t really show us much of the full picture. Sure, such numbers might tell us who has the most highlight hits on their resume, but they give us little perspective on what linemen are helping their team the most overall. Put more bluntly, it undervalues the "playmakers" of the defensive line. Any defensive linemen can reach the quarterback when unblocked, and similarly any defensive linemen that draws a double block every play can be stunted. What is imminently important is not necessarily who draws the most pressure or the most sacks, but instead who helps his team's chances of play disruption most, given where on the field he is and how many blockers are present. 

*The IDPI metric is centered around the idea that, on any given play, a rusher improves, or hurts, his entire line’s chance of achieving a pass disruption by some probability. This probability, if intuitively estimated, can serve as an measurement of how effective a defensive linemen is.*


### i. Model Selection

The dynamics of the pass rush are certainly team-oriented. As offensive and defensive linemen clash on the line of scrimmage, blockers are beaten and switch to different rushers, and rushers are met with spatial obstructions beyond their first defender. As a result, instead of modeling the probability of individual rushers disrupting pass plays, we found it to be much more appropriate to consider a team’s probability of disrupting a pass plays cohesively at each point of time in a play. 

A one layer long-short-term-memory (LSTM) model was implimented for time series classification at various points after the ball snap for the binary play disruption outcome. Note that play disruption is defined as any type of hit, hurry, or sack recorded by the defense over the course of a play. Typically, LSTM models for time series classification are only trained and backpropagated according to loss in the final fully connected layer (i.e., usually the entire time series is of one class). For our purposes, though, we need an alternative method which trains the probabilities of a play disruption at all points in time during a play. Consequently, this specific LSTM model is trained on the equally weighted sum of SoftMax outputs in the hidden layers of the model. As a result, probabilities of play disruption can be predicted at each time point in the play. 

Each tenth of a second after the snap, a cross-entropy loss value is calculated for the prediction of true play disruption, or lack thereof. The total of these losses is used for backpropagation. Note that our model is trained up to five full seconds after the play snap, and shorter plays are zero-padded to accommodate consistent play length.

<div>
<center><img src="https://imgur.com/CWXx6mY.png" width="1000px;"/><center>
</div>

### ii. Feature Selection & Engineering


Modeling a team's probability of pass disruption requires feature engineering that utilizes individual player spatial data at various time points throughout the play. Additionally, engineered features should be ideal and intuitive for accommodating the replacement of player data with situational averages in our IDPI metric creation pipeline. Below are the selected team-level features that were fed into the LSTM model. In order to make such team level features a direct extension of individual rusher features, summary statistics of player feature values were used. 

<div>
<center><img src="https://imgur.com/Jy1kvJv.gif" width="600px;"/><center>
</div>

Of particular interest are the features for "Blockers Remaining," "Minimum Rusher Distance to QB," "Maximum Rusher Distance Gained Toward QB," and "Maximum Rusher Acceleration toward QB." Such features are created with the idea that they highlight best rusher on the field. Note that the number of blockers remaining is calculated using (1) the number of encountered blockers on the play (2) whether the rusher was closer to the QB than those blockers. Resulting LSTM models are trained on each time point prediction **achieve an overall AUC of 0.7 with increasing AUC values between ball snap (0.57 AUC) to end of play (0.85 AUC) on validation sets.**

    
<div>
<center><img src="https://imgur.com/7PGhTTv.png" width="800px;"/><center>
</div>
    




### iii. Replacement of Individual Pass Rusher Data with Situational Averages

Collecting situational rusher averages over various times after the snap is an essential step in the creation of the IDPI metric. Defensive linemen need to be compared to rushers in the same "start-of-play" situation in order to assess general performance. **Such "start-of-play" situational average groupings are defined by (1) the number of blockers a rusher encounters over the course of the play and (2) the position of the rusher at the ball snap** (i.e., Defensive Back, Defensive End, etc.). Using these groupings, average individual level features ("Number of Blockers Remaining", "Distance Gained to QB", "Acceleration Toward QB", etc.) are collected and utilized to replace players in true-play situations. 

For each play, and for each active pass rusher, team play disruption probabilities are recalculated with rushers replaced with their respective situational averages. These probabilities are compared to the true play probabilities to get our IDPI metric.


<div>
<center><img src="https://imgur.com/kwgoX5B.png" width="600px;"/><center>
</div>

# Metric Creation

### i. IDPI Metric

Metrics can be collected cross-sectionally at any given instant during a play, as shown above. Such metrics represent the amount a player increases his team’s probability of play disruption compared to the average linemen in a similar situation at a particular time point after the snap. 

Below are plots of instantaneous IDPI tracking over the course of plays resulting in disruptive or non-disruption outcomes. Players that are about to create an individual disruption clearly have higher IDPI values, but even more notably, we see high IDPI metric assignment for players performing well regardless of being close to an individual disruption. Identifying players that perform well regardless of achieving individual disruptions is paramount for the goals of the IDPI metric.

<table style='margin: 0 auto'>
<tr>
<th style="width:400px;">Disruption Example</th>
<th style="width:400px;">Non-Disruption Example</th>
<tr>
<td> <img src="https://imgur.com/X2MsweX.gif" alt="Drawing" style="width: 400px;"/> </td>
<td> <img src="https://imgur.com/b2QzCS6.gif" alt="Drawing" style="width: 400px;"/> </td>
<tr>
<td> <img src="https://imgur.com/5C1NCSH.gif" alt="Drawing" style="width: 400px;"/> </td>
<td> <img src="https://imgur.com/agOTGPP.gif" alt="Drawing" style="width: 400px;"/> </td>
</tr>
</table>

### ii. Play-Long & Season-Long IDPI Metric

Play and season long rusher metrics can also be calculated as an average of a player’s IDPI over time. Using these play and season long metrics, players can be ranked in best general IDPI performance. 

Below are the top rushers according to average IDPI over the first 8 weeks of the 2021-2022 NFL season, conditional on players having 50 or more active plays. As expected, many of the anticipated top rushers in the league (i.e., Jeffery Simmons, Aaron Donald, etc.) emerge to the top of the rankings, but players that may not get as much publicity or play time (i.e., Bravvion Roy) remain in the top 20. The uniqueness of the IDPI and its ability to identify pass rushing playmakers regardless of individual disruption frequency allow us to identify these less talked about players, who should certainly be played more often and more highly valued. IDPI also allows us to assess the value of players that have been injured for most of our data period (i.e., Marcus Davenport) compared to current overall NFL pass rushing standard.

It is also evident that, while there is a relationship between the number of individual disruptions per play and the IDPI metric, the highest ranked players are not always those with the most individual disruptions. This is expected, as IDPI instead highly values players that increase their teams *overall* chance of play disruption.

<center>
<table class="gt_table">

<thead class="gt_col_headings">
<tr>
<th class="gt_col_heading gt_columns_bottom_border gt_left" rowspan="1" colspan="1">Name</th>
<th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1">Rank</th>
<th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1">IDPI Metric</th>
<th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1">Number of Active Plays</th>
<th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1">Weight (lb)</th>
<th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1">Height (In)</th>
<th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1">Individual Play Disruptions</th>
<th class="gt_col_heading gt_columns_bottom_border gt_right" rowspan="1" colspan="1">Individual Disruptions per Play</th>
</tr>
</thead>
<tbody class="gt_table_body">
<tr><td class="gt_row gt_left">Jeffery Simmons</td>
<td class="gt_row gt_right">1</td>
<td class="gt_row gt_right" style="background-color: #0D47A1; color: #FFFFFF;">6.305</td>
<td class="gt_row gt_right" style="background-color: #0D47A1; color: #FFFFFF;">262</td>
<td class="gt_row gt_right" style="background-color: #2A99F3; color: #000000;">300</td>
<td class="gt_row gt_right" style="background-color: #1E88E5; color: #FFFFFF;">76</td>
<td class="gt_row gt_right" style="background-color: #1F8BE8; color: #FFFFFF;">34</td>
<td class="gt_row gt_right" style="background-color: #5FB2F6; color: #000000;">0.13</td></tr>
<tr><td class="gt_row gt_left">Aaron Donald</td>
<td class="gt_row gt_right">2</td>
<td class="gt_row gt_right" style="background-color: #1D84E1; color: #FFFFFF;">5.855</td>
<td class="gt_row gt_right" style="background-color: #0F4CA6; color: #FFFFFF;">258</td>
<td class="gt_row gt_right" style="background-color: #5FB2F6; color: #000000;">280</td>
<td class="gt_row gt_right" style="background-color: #A6D4FA; color: #000000;">73</td>
<td class="gt_row gt_right" style="background-color: #1B7DDA; color: #FFFFFF;">38</td>
<td class="gt_row gt_right" style="background-color: #3CA1F5; color: #000000;">0.15</td></tr>
<tr><td class="gt_row gt_left">Marcus Davenport</td>
<td class="gt_row gt_right">3</td>
<td class="gt_row gt_right" style="background-color: #39A0F4; color: #000000;">5.547</td>
<td class="gt_row gt_right" style="background-color: #DCEFFD; color: #000000;">58</td>
<td class="gt_row gt_right" style="background-color: #93CBF9; color: #000000;">265</td>
<td class="gt_row gt_right" style="background-color: #0D47A1; color: #FFFFFF;">78</td>
<td class="gt_row gt_right" style="background-color: #ABD6FA; color: #000000;">12</td>
<td class="gt_row gt_right" style="background-color: #176FCB; color: #FFFFFF;">0.21</td></tr>
<tr><td class="gt_row gt_left">Myles Garrett</td>
<td class="gt_row gt_right">4</td>
<td class="gt_row gt_right" style="background-color: #4AA8F5; color: #000000;">5.464</td>
<td class="gt_row gt_right" style="background-color: #1B7CD8; color: #FFFFFF;">208</td>
<td class="gt_row gt_right" style="background-color: #7BBFF7; color: #000000;">272</td>
<td class="gt_row gt_right" style="background-color: #1E88E5; color: #FFFFFF;">76</td>
<td class="gt_row gt_right" style="background-color: #0D47A1; color: #FFFFFF;">50</td>
<td class="gt_row gt_right" style="background-color: #0D47A1; color: #FFFFFF;">0.24</td></tr>
<tr><td class="gt_row gt_left">Christian Barmore</td>
<td class="gt_row gt_right">5</td>
<td class="gt_row gt_right" style="background-color: #51ABF5; color: #000000;">5.429</td>
<td class="gt_row gt_right" style="background-color: #1F8DEA; color: #FFFFFF;">185</td>
<td class="gt_row gt_right" style="background-color: #1F8EEB; color: #FFFFFF;">310</td>
<td class="gt_row gt_right" style="background-color: #176DC9; color: #FFFFFF;">77</td>
<td class="gt_row gt_right" style="background-color: #51ABF5; color: #000000;">23</td>
<td class="gt_row gt_right" style="background-color: #75BCF7; color: #000000;">0.12</td></tr>
<tr><td class="gt_row gt_left">Maliek Collins</td>
<td class="gt_row gt_right">6</td>
<td class="gt_row gt_right" style="background-color: #52ACF5; color: #000000;">5.426</td>
<td class="gt_row gt_right" style="background-color: #4EAAF5; color: #000000;">139</td>
<td class="gt_row gt_right" style="background-color: #2090ED; color: #000000;">308</td>
<td class="gt_row gt_right" style="background-color: #64B5F6; color: #000000;">74</td>
<td class="gt_row gt_right" style="background-color: #80C2F8; color: #000000;">17</td>
<td class="gt_row gt_right" style="background-color: #75BCF7; color: #000000;">0.12</td></tr>
<tr><td class="gt_row gt_left">Julian Okwara</td>
<td class="gt_row gt_right">7</td>
<td class="gt_row gt_right" style="background-color: #69B7F6; color: #000000;">5.316</td>
<td class="gt_row gt_right" style="background-color: #D0E8FC; color: #000000;">65</td>
<td class="gt_row gt_right" style="background-color: #E3F2FD; color: #000000;">241</td>
<td class="gt_row gt_right" style="background-color: #176DC9; color: #FFFFFF;">77</td>
<td class="gt_row gt_right" style="background-color: #ABD6FA; color: #000000;">12</td>
<td class="gt_row gt_right" style="background-color: #1F8AE7; color: #FFFFFF;">0.18</td></tr>
<tr><td class="gt_row gt_left">Akiem Hicks</td>
<td class="gt_row gt_right">8</td>
<td class="gt_row gt_right" style="background-color: #86C5F8; color: #000000;">5.210</td>
<td class="gt_row gt_right" style="background-color: #94CCF9; color: #000000;">98</td>
<td class="gt_row gt_right" style="background-color: #0D47A1; color: #FFFFFF;">352</td>
<td class="gt_row gt_right" style="background-color: #1E88E5; color: #FFFFFF;">76</td>
<td class="gt_row gt_right" style="background-color: #BBDEFB; color: #000000;">10</td>
<td class="gt_row gt_right" style="background-color: #A2D2FA; color: #000000;">0.10</td></tr>
<tr><td class="gt_row gt_left">Linval Joseph</td>
<td class="gt_row gt_right">9</td>
<td class="gt_row gt_right" style="background-color: #89C6F8; color: #000000;">5.200</td>
<td class="gt_row gt_right" style="background-color: #71BBF7; color: #000000;">117</td>
<td class="gt_row gt_right" style="background-color: #1974D0; color: #FFFFFF;">329</td>
<td class="gt_row gt_right" style="background-color: #1E88E5; color: #FFFFFF;">76</td>
<td class="gt_row gt_right" style="background-color: #A2D2FA; color: #000000;">13</td>
<td class="gt_row gt_right" style="background-color: #8BC7F9; color: #000000;">0.11</td></tr>
<tr><td class="gt_row gt_left">Javon Hargrave</td>
<td class="gt_row gt_right">10</td>
<td class="gt_row gt_right" style="background-color: #8AC7F9; color: #000000;">5.193</td>
<td class="gt_row gt_right" style="background-color: #2296F3; color: #000000;">169</td>
<td class="gt_row gt_right" style="background-color: #2093F0; color: #000000;">305</td>
<td class="gt_row gt_right" style="background-color: #64B5F6; color: #000000;">74</td>
<td class="gt_row gt_right" style="background-color: #3DA2F5; color: #000000;">26</td>
<td class="gt_row gt_right" style="background-color: #3CA1F5; color: #000000;">0.15</td></tr>
<tr><td class="gt_row gt_left">Kenny Clark</td>
<td class="gt_row gt_right">11</td>
<td class="gt_row gt_right" style="background-color: #99CEF9; color: #000000;">5.138</td>
<td class="gt_row gt_right" style="background-color: #1566C1; color: #FFFFFF;">238</td>
<td class="gt_row gt_right" style="background-color: #1E89E6; color: #FFFFFF;">314</td>
<td class="gt_row gt_right" style="background-color: #349DF4; color: #000000;">75</td>
<td class="gt_row gt_right" style="background-color: #1E88E5; color: #FFFFFF;">35</td>
<td class="gt_row gt_right" style="background-color: #3CA1F5; color: #000000;">0.15</td></tr>
<tr><td class="gt_row gt_left">Dexter Lawrence</td>
<td class="gt_row gt_right">12</td>
<td class="gt_row gt_right" style="background-color: #A1D1FA; color: #000000;">5.109</td>
<td class="gt_row gt_right" style="background-color: #2195F2; color: #000000;">172</td>
<td class="gt_row gt_right" style="background-color: #145FBA; color: #FFFFFF;">342</td>
<td class="gt_row gt_right" style="background-color: #1E88E5; color: #FFFFFF;">76</td>
<td class="gt_row gt_right" style="background-color: #58AFF6; color: #000000;">22</td>
<td class="gt_row gt_right" style="background-color: #5FB2F6; color: #000000;">0.13</td></tr>
<tr><td class="gt_row gt_left">Bravvion Roy</td>
<td class="gt_row gt_right">13</td>
<td class="gt_row gt_right" style="background-color: #ACD7FA; color: #000000;">5.066</td>
<td class="gt_row gt_right" style="background-color: #E3F2FD; color: #000000;">54</td>
<td class="gt_row gt_right" style="background-color: #176ECA; color: #FFFFFF;">333</td>
<td class="gt_row gt_right" style="background-color: #A6D4FA; color: #000000;">73</td>
<td class="gt_row gt_right" style="background-color: #E3F2FD; color: #000000;">5</td>
<td class="gt_row gt_right" style="background-color: #B9DDFB; color: #000000;">0.09</td></tr>
<tr><td class="gt_row gt_left">Takkarist McKinley</td>
<td class="gt_row gt_right">14</td>
<td class="gt_row gt_right" style="background-color: #B9DDFB; color: #000000;">5.015</td>
<td class="gt_row gt_right" style="background-color: #6BB8F6; color: #000000;">120</td>
<td class="gt_row gt_right" style="background-color: #93CBF9; color: #000000;">265</td>
<td class="gt_row gt_right" style="background-color: #64B5F6; color: #000000;">74</td>
<td class="gt_row gt_right" style="background-color: #90CAF9; color: #000000;">15</td>
<td class="gt_row gt_right" style="background-color: #75BCF7; color: #000000;">0.12</td></tr>
<tr><td class="gt_row gt_left">Grady Jarrett</td>
<td class="gt_row gt_right">15</td>
<td class="gt_row gt_right" style="background-color: #C9E5FC; color: #000000;">4.954</td>
<td class="gt_row gt_right" style="background-color: #1F8DEA; color: #FFFFFF;">185</td>
<td class="gt_row gt_right" style="background-color: #2093F0; color: #000000;">305</td>
<td class="gt_row gt_right" style="background-color: #E3F2FD; color: #000000;">72</td>
<td class="gt_row gt_right" style="background-color: #5EB2F6; color: #000000;">21</td>
<td class="gt_row gt_right" style="background-color: #8BC7F9; color: #000000;">0.11</td></tr>
<tr><td class="gt_row gt_left">Adam Gotsis</td>
<td class="gt_row gt_right">16</td>
<td class="gt_row gt_right" style="background-color: #D2E9FC; color: #000000;">4.917</td>
<td class="gt_row gt_right" style="background-color: #A4D3FA; color: #000000;">90</td>
<td class="gt_row gt_right" style="background-color: #4DA9F5; color: #000000;">287</td>
<td class="gt_row gt_right" style="background-color: #1E88E5; color: #FFFFFF;">76</td>
<td class="gt_row gt_right" style="background-color: #D3EAFC; color: #000000;">7</td>
<td class="gt_row gt_right" style="background-color: #CEE7FC; color: #000000;">0.08</td></tr>
<tr><td class="gt_row gt_left">Mario Edwards</td>
<td class="gt_row gt_right">17</td>
<td class="gt_row gt_right" style="background-color: #D5EBFC; color: #000000;">4.906</td>
<td class="gt_row gt_right" style="background-color: #D4EAFC; color: #000000;">63</td>
<td class="gt_row gt_right" style="background-color: #5FB2F6; color: #000000;">280</td>
<td class="gt_row gt_right" style="background-color: #349DF4; color: #000000;">75</td>
<td class="gt_row gt_right" style="background-color: #DBEEFD; color: #000000;">6</td>
<td class="gt_row gt_right" style="background-color: #A2D2FA; color: #000000;">0.10</td></tr>
<tr><td class="gt_row gt_left">J.J. Watt</td>
<td class="gt_row gt_right">18</td>
<td class="gt_row gt_right" style="background-color: #DDEFFD; color: #000000;">4.872</td>
<td class="gt_row gt_right" style="background-color: #1F8BE8; color: #FFFFFF;">188</td>
<td class="gt_row gt_right" style="background-color: #4AA8F5; color: #000000;">288</td>
<td class="gt_row gt_right" style="background-color: #176DC9; color: #FFFFFF;">77</td>
<td class="gt_row gt_right" style="background-color: #379FF4; color: #000000;">27</td>
<td class="gt_row gt_right" style="background-color: #4DAAF5; color: #000000;">0.14</td></tr>
<tr><td class="gt_row gt_left">Grover Stewart</td>
<td class="gt_row gt_right">19</td>
<td class="gt_row gt_right" style="background-color: #E2F2FD; color: #000000;">4.850</td>
<td class="gt_row gt_right" style="background-color: #53ACF5; color: #000000;">136</td>
<td class="gt_row gt_right" style="background-color: #1E88E5; color: #FFFFFF;">315</td>
<td class="gt_row gt_right" style="background-color: #1E88E5; color: #FFFFFF;">76</td>
<td class="gt_row gt_right" style="background-color: #BBDEFB; color: #000000;">10</td>
<td class="gt_row gt_right" style="background-color: #E3F2FD; color: #000000;">0.07</td></tr>
<tr><td class="gt_row gt_left">Cameron Heyward</td>
<td class="gt_row gt_right">20</td>
<td class="gt_row gt_right" style="background-color: #E3F2FD; color: #000000;">4.846</td>
<td class="gt_row gt_right" style="background-color: #1D84E1; color: #FFFFFF;">198</td>
<td class="gt_row gt_right" style="background-color: #389FF4; color: #000000;">295</td>
<td class="gt_row gt_right" style="background-color: #176DC9; color: #FFFFFF;">77</td>
<td class="gt_row gt_right" style="background-color: #379FF4; color: #000000;">27</td>
<td class="gt_row gt_right" style="background-color: #4DAAF5; color: #000000;">0.14</td></tr>
</tbody>
</table>

# Discussion

The IDPI metric innovates an entirely new way to assess pass rushers. The defensive line functions as a whole, and members should be assessed with this in mind. IDPI identifies players, in a variety of situations, who exhibit above average ability in improving team likliehood of play disruption. Focusing on individual rusher play disruptions captures only a part of the picture. The intricacies of the IDPI metric would allow coaches and analysts to identify high-achieving and often overlooked players, while continuing to recognize big-name players with the most entertaining highlight plays.

Future work aims to refine situational average collection for player replacement, potentially adding more averaging groups to capture true spatial time series distributions. Additionally, creating player metric tweaks for rushers with low active play sample sizes would be ideal for moving toward identifying more players that see less of the field. 

# Code

Review code [here on GitHub](https://github.com/nickb1125/Big-Data-Bowl-Linemen).